# Calculate initial opportunity and competition level score

Goal:
* Make an easy score for the product demo

### Import and initialize

In [1]:
import arcgis
from arcgis.gis import GIS
from arcgis.features import FeatureLayer, FeatureLayerCollection

import pandas as pd

import psycopg2

import sys
sys.path.append('../../')
from utils import get_config

sys.path.append('../')
from gis_resources import san_diego_county_zips,  execute_sql, create_where_clause, read_exact_food_biz_categories, read_exact_unhealthy_food_biz_categories

In [2]:
username = get_config("arcgis","username")
password = get_config("arcgis","passkey")
gis = GIS("https://ucsdonline.maps.arcgis.com/home", username=username, password=password)

Definition a function which takes in below variables and give an opportunity score.

### Query feature layers into dataframes

#### Consumer Spending Layer

In [3]:
consumer_spending_layer_a = FeatureLayer(url="https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/a2afc4/FeatureServer/0")
consumer_spending_layer_a

<FeatureLayer url:"https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/a2afc4/FeatureServer/0">

In [4]:
# Picking up sample variables for just poc on rendering maps as per our need.
csa_out_fields = ["fips", "X1002_A", "X1130_A", "X1130FY_A"]

In [5]:
consumer_spending_layer_a_df = consumer_spending_layer_a.query(out_fields=csa_out_fields,
                              as_df=True,
                              return_geometry=False)

del consumer_spending_layer_a_df['FID']

consumer_spending_layer_a_df

,fips,x1002_a,x1130_a,x1130fy_a
0,060730100101,6877.52,2942.05,3623.67
1,060730100102,8364.19,3457.18,4231.38
2,060730100103,7101.43,2935.24,3277.84
3,060730100111,10566.02,4519.91,5287.83
4,060730150012,11587.27,4781.76,6792.55
...,...,...,...,...
2052,060730221012,20195.98,8544.28,9033.19
2053,060730221021,22616.71,9568.42,11358.21
2054,060730221022,22722.96,9613.37,11373.74
2055,060730221023,14510.63,6183.54,7345.41


#### Business Data Layer

Note: S01_BUS and N01_BUS gave the same number, so S01_BUS was used.

In [6]:
business_variables_layer = FeatureLayer(url="https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/a633a0/FeatureServer/0")
business_variables_layer

<FeatureLayer url:"https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/a633a0/FeatureServer/0">

In [7]:
business_out_fields = ["fips", "S01_BUS", "S12_BUS", "S16_BUS", "N13_BUS", "N37_BUS"]

In [8]:
business_variables_layer_df = business_variables_layer.query(out_fields=business_out_fields,
                              as_df=True,
                              return_geometry=False)

del business_variables_layer_df['FID']

business_variables_layer_df

,fips,n13_bus,n37_bus,s01_bus,s12_bus,s16_bus
0,060730083111,0.0,0.0,47.0,0.0,0.0
1,060730083112,0.0,0.0,25.0,0.0,0.0
2,060730083121,0.0,12.0,101.0,0.0,12.0
3,060730083122,0.0,6.0,35.0,0.0,6.0
4,060730083123,0.0,0.0,17.0,0.0,0.0
...,...,...,...,...,...,...
2052,060730221012,8.0,41.0,1594.0,12.0,40.0
2053,060730221021,0.0,0.0,36.0,1.0,0.0
2054,060730221022,4.0,17.0,180.0,5.0,16.0
2055,060730221023,0.0,0.0,24.0,0.0,0.0


#### Demographics Layer
Note on Diversity Index (DIVINDX_CY): https://storymaps.arcgis.com/stories/94db3c9e75b54e22a0e99978ad77df54

In [9]:
demographics_layer_1 = FeatureLayer(url="https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/aff1b6/FeatureServer/0")
demographics_layer_1

<FeatureLayer url:"https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/aff1b6/FeatureServer/0">

In [10]:
demographics_layer_1_out_fields = ['fips', 'TOTPOP_CY', "POPDENS_CY", "POPDENS_FY", "MEDDI_CY", "DIVINDX_CY"]

In [11]:
demographics_layer_1_df = demographics_layer_1.query(out_fields=demographics_layer_1_out_fields,
                              as_df=True,
                              return_geometry=False)
del demographics_layer_1_df['FID']

demographics_layer_1_df

,divindx_cy,fips,meddi_cy,popdens_cy,popdens_fy,totpop_cy
0,46.0,060730001001,129998.0,4714.0,4690.4,1199.0
1,45.6,060730001002,152077.0,4993.9,4864.1,1692.0
2,47.5,060730002011,100616.0,4614.1,4793.2,902.0
3,59.9,060730002021,60616.0,8342.2,8218.7,1283.0
4,62.4,060730002022,45921.0,6809.9,6742.7,911.0
...,...,...,...,...,...,...
2052,58.4,060730221012,113615.0,131.2,129.1,450.0
2053,52.9,060730221021,131072.0,3204.1,3326.8,1802.0
2054,64.2,060730221022,132157.0,3840.9,4090.8,1906.0
2055,80.0,060730221023,87061.0,4379.3,4526.5,2171.0


In [12]:
demographics_layer_2 = FeatureLayer(url="https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/ab8ff7/FeatureServer/0")
demographics_layer_2

<FeatureLayer url:"https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/ab8ff7/FeatureServer/0">

In [13]:
demographics_layer_2_out_fields = ['fips', "TOTPOP_FY"]

In [14]:
demographics_layer_2_df = demographics_layer_2.query(out_fields=demographics_layer_2_out_fields,
                              as_df=True,
                              return_geometry=False)
del demographics_layer_2_df['FID']

demographics_layer_2_df

,fips,totpop_fy
0,060730083111,1821.0
1,060730083112,1130.0
2,060730083121,611.0
3,060730083122,1419.0
4,060730083123,769.0
...,...,...
2052,060730221012,443.0
2053,060730221021,1871.0
2054,060730221022,2030.0
2055,060730221023,2244.0


### Merge the feature layers together

In [15]:
opportunity_competition_df = consumer_spending_layer_a_df.merge(business_variables_layer_df, left_on = 'fips', right_on = 'fips')
opportunity_competition_df = opportunity_competition_df.merge(demographics_layer_1_df, left_on = 'fips', right_on = 'fips')
opportunity_competition_df = opportunity_competition_df.merge(demographics_layer_2_df, left_on = 'fips', right_on = 'fips')
opportunity_competition_df

,fips,x1002_a,x1130_a,x1130fy_a,n13_bus,n37_bus,s01_bus,s12_bus,s16_bus,divindx_cy,meddi_cy,popdens_cy,popdens_fy,totpop_cy,totpop_fy
0,060730100101,6877.52,2942.05,3623.67,0.0,0.0,11.0,0.0,0.0,78.7,47338.0,15154.6,14831.8,1690.0,1654.0
1,060730100102,8364.19,3457.18,4231.38,2.0,3.0,25.0,2.0,2.0,80.8,45231.0,9018.8,8816.6,2185.0,2136.0
2,060730100103,7101.43,2935.24,3277.84,1.0,3.0,29.0,2.0,3.0,84.8,46459.0,15980.3,15622.9,1565.0,1530.0
3,060730100111,10566.02,4519.91,5287.83,1.0,1.0,16.0,1.0,1.0,84.2,75000.0,7335.9,7390.2,1485.0,1496.0
4,060730150012,11587.27,4781.76,6792.55,2.0,8.0,43.0,1.0,8.0,73.1,59813.0,9046.8,8844.2,1563.0,1528.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2052,060730221012,20195.98,8544.28,9033.19,8.0,41.0,1594.0,12.0,40.0,58.4,113615.0,131.2,129.1,450.0,443.0
2053,060730221021,22616.71,9568.42,11358.21,0.0,0.0,36.0,1.0,0.0,52.9,131072.0,3204.1,3326.8,1802.0,1871.0
2054,060730221022,22722.96,9613.37,11373.74,4.0,17.0,180.0,5.0,16.0,64.2,132157.0,3840.9,4090.8,1906.0,2030.0
2055,060730221023,14510.63,6183.54,7345.41,0.0,0.0,24.0,0.0,0.0,80.0,87061.0,4379.3,4526.5,2171.0,2244.0


In [16]:
var_mapping = {'x1002_a': 'Avg 2022 Food', 'x1003_a': 'Avg 2022 Food at Home', 'x1130_a': 'Avg 2022 Food Away from Home', 'x1002fy_a': 'Avg 2027 Food', 'x1003fy_a': 'Avg 2027 Food at Home', 'x1130fy_a': 'Avg 2027 Food Away from Home', 's01_bus': '2021 Total (SIC01-99) Businesses', 's12_bus': '2021 Food Stores (SIC54) Businesses', 's16_bus': '2021 Eating & Drinking (SIC58) Businesses', 'n01_bus': '2021 Total (NAICS11-99) Businesses', 'n13_bus': '2021 Food & Beverage Stores (NAICS445) Businesses', 'n37_bus': '2021 Food Srv & Drinking Places (NAICS722) Businesses', 'mp28068a_b': '2021 Eat Healthy/Pay Attention to Nutrition: 1-Disagree Completely', 'mp28068a_i': '2021 Eat Healthy/Pay Attention to Nutrition: 1-Disagree Completely: Index', 'mp28069a_b': '2021 Eat Healthy/Pay Attention to Nutrition: 2-Disagree Somewhat', 'mp28069a_i': '2021 Eat Healthy/Pay Attention to Nutrition: 2-Disagree Somewhat: Index', 'mp28070a_b': '2021 Eat Healthy/Pay Attention to Nutrition: 3-Agree Somewhat', 'mp28070a_i': '2021 Eat Healthy/Pay Attention to Nutrition: 3-Agree Somewhat: Index', 'mp28071a_b': '2021 Eat Healthy/Pay Attention to Nutrition: 4-Agree Completely', 'mp28071a_i': '2021 Eat Healthy/Pay Attention to Nutrition: 4-Agree Completely: Index', 'mp28258a_b': '2021 Like Healthier Options at Fast Food: 1-Disagree Completely', 'mp28258a_i': '2021 Like Healthier Options at Fast Food: 1-Disagree Completely: Index', 'mp28259a_b': '2021 Like Healthier Options at Fast Food: 2-Disagree Somewhat', 'mp28259a_i': '2021 Like Healthier Options at Fast Food: 2-Disagree Somewhat: Index', 'mp28260a_b': '2021 Like Healthier Options at Fast Food: 3-Agree Somewhat', 'mp28260a_i': '2021 Like Healthier Options at Fast Food: 3-Agree Somewhat: Index', 'mp28261a_b': '2021 Like Healthier Options at Fast Food: 4-Agree Completely', 'mp28261a_i': '2021 Like Healthier Options at Fast Food: 4-Agree Completely: Index', 'totpop_cy': '2021 Total Population (Esri)', 'popdens_cy': '2021 Population Density (Pop per Square Mile) (Esri)', 'totpop_fy': '2026 Total Population (Esri)', 'popdens_fy': '2026 Population Density (Pop per Square Mile) (Esri)', 'meddi_cy': '2021 Median Disposable Income (Esri)'
              , 'divindx_cy': '2021 Diversity Index (Esri)', 'divindx_fy': '2026 Diversity Index (Esri)'}
var_mapping

{'x1002_a': 'Avg 2022 Food',
 'x1003_a': 'Avg 2022 Food at Home',
 'x1130_a': 'Avg 2022 Food Away from Home',
 'x1002fy_a': 'Avg 2027 Food',
 'x1003fy_a': 'Avg 2027 Food at Home',
 'x1130fy_a': 'Avg 2027 Food Away from Home',
 's01_bus': '2021 Total (SIC01-99) Businesses',
 's12_bus': '2021 Food Stores (SIC54) Businesses',
 's16_bus': '2021 Eating & Drinking (SIC58) Businesses',
 'n01_bus': '2021 Total (NAICS11-99) Businesses',
 'n13_bus': '2021 Food & Beverage Stores (NAICS445) Businesses',
 'n37_bus': '2021 Food Srv & Drinking Places (NAICS722) Businesses',
 'mp28068a_b': '2021 Eat Healthy/Pay Attention to Nutrition: 1-Disagree Completely',
 'mp28068a_i': '2021 Eat Healthy/Pay Attention to Nutrition: 1-Disagree Completely: Index',
 'mp28069a_b': '2021 Eat Healthy/Pay Attention to Nutrition: 2-Disagree Somewhat',
 'mp28069a_i': '2021 Eat Healthy/Pay Attention to Nutrition: 2-Disagree Somewhat: Index',
 'mp28070a_b': '2021 Eat Healthy/Pay Attention to Nutrition: 3-Agree Somewhat',
 'mp

In [17]:
opportunity_competition_df.rename(columns = var_mapping, inplace = True)
opportunity_competition_df

,fips,Avg 2022 Food,Avg 2022 Food Away from Home,Avg 2027 Food Away from Home,2021 Food & Beverage Stores (NAICS445) Businesses,2021 Food Srv & Drinking Places (NAICS722) Businesses,2021 Total (SIC01-99) Businesses,2021 Food Stores (SIC54) Businesses,2021 Eating & Drinking (SIC58) Businesses,2021 Diversity Index (Esri),2021 Median Disposable Income (Esri),2021 Population Density (Pop per Square Mile) (Esri),2026 Population Density (Pop per Square Mile) (Esri),2021 Total Population (Esri),2026 Total Population (Esri)
0,060730100101,6877.52,2942.05,3623.67,0.0,0.0,11.0,0.0,0.0,78.7,47338.0,15154.6,14831.8,1690.0,1654.0
1,060730100102,8364.19,3457.18,4231.38,2.0,3.0,25.0,2.0,2.0,80.8,45231.0,9018.8,8816.6,2185.0,2136.0
2,060730100103,7101.43,2935.24,3277.84,1.0,3.0,29.0,2.0,3.0,84.8,46459.0,15980.3,15622.9,1565.0,1530.0
3,060730100111,10566.02,4519.91,5287.83,1.0,1.0,16.0,1.0,1.0,84.2,75000.0,7335.9,7390.2,1485.0,1496.0
4,060730150012,11587.27,4781.76,6792.55,2.0,8.0,43.0,1.0,8.0,73.1,59813.0,9046.8,8844.2,1563.0,1528.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2052,060730221012,20195.98,8544.28,9033.19,8.0,41.0,1594.0,12.0,40.0,58.4,113615.0,131.2,129.1,450.0,443.0
2053,060730221021,22616.71,9568.42,11358.21,0.0,0.0,36.0,1.0,0.0,52.9,131072.0,3204.1,3326.8,1802.0,1871.0
2054,060730221022,22722.96,9613.37,11373.74,4.0,17.0,180.0,5.0,16.0,64.2,132157.0,3840.9,4090.8,1906.0,2030.0
2055,060730221023,14510.63,6183.54,7345.41,0.0,0.0,24.0,0.0,0.0,80.0,87061.0,4379.3,4526.5,2171.0,2244.0


In [18]:
opportunity_competition_df_og = opportunity_competition_df.copy(deep = True)

#### Create variables

Perecentage of spending on food away from home per household. Calculated by dividing Avg spent on food away from home by total spent on food. 

In [19]:
opportunity_competition_df["Food Away From Home Spending Rate"] = (opportunity_competition_df["Avg 2022 Food Away from Home"] 
                                                                   / opportunity_competition_df["Avg 2022 Food"])
opportunity_competition_df["Food Away From Home Spending Rate"]

0       0.427778
1       0.413331
2       0.413331
3       0.427778
4       0.412674
          ...   
2052    0.423068
2053    0.423069
2054    0.423069
2055    0.426139
2056    0.426138
Name: Food Away From Home Spending Rate, Length: 2057, dtype: Float64

Projected spending growth on food away from home 22-27 per household. Calculated by dividing 2027 projected spending on food away from home by the 2022 actual values.

In [20]:
opportunity_competition_df["Project 5Yr Food Away Home Spending Growth"] = (opportunity_competition_df["Avg 2027 Food Away from Home"] / 
                                                                  opportunity_competition_df["Avg 2022 Food Away from Home"] -1)

opportunity_competition_df["Project 5Yr Food Away Home Spending Growth"]

0       0.231682
1        0.22394
2        0.11672
3       0.169897
4       0.420513
          ...   
2052    0.057221
2053    0.187052
2054    0.183117
2055    0.187897
2056    0.193864
Name: Project 5Yr Food Away Home Spending Growth, Length: 2057, dtype: Float64

Total % of food related business. Calculated by dividing the number of food businesses in the block group by the total number of businesses. 

In [21]:
opportunity_competition_df["Total Pct of Food Related Businesses"] = (opportunity_competition_df["2021 Food & Beverage Stores (NAICS445) Businesses"] + 
 opportunity_competition_df["2021 Food Srv & Drinking Places (NAICS722) Businesses"] + 
 opportunity_competition_df["2021 Food Stores (SIC54) Businesses"] + 
 opportunity_competition_df["2021 Eating & Drinking (SIC58) Businesses"]) / opportunity_competition_df["2021 Total (SIC01-99) Businesses"]

opportunity_competition_df["Total Pct of Food Related Businesses"]

0            0.0
1           0.36
2       0.310345
3           0.25
4        0.44186
          ...   
2052    0.063363
2053    0.027778
2054    0.233333
2055         0.0
2056    0.069486
Name: Total Pct of Food Related Businesses, Length: 2057, dtype: Float64

Total Food Business Per capita 21. Calculated by dividing the total number of food related businesses by the total population in the block group. 

In [22]:
opportunity_competition_df["Pct of Food Related Businesses Per Capita"] = (opportunity_competition_df["2021 Food & Beverage Stores (NAICS445) Businesses"] + 
 opportunity_competition_df["2021 Food Srv & Drinking Places (NAICS722) Businesses"] + 
 opportunity_competition_df["2021 Food Stores (SIC54) Businesses"] + 
 opportunity_competition_df["2021 Eating & Drinking (SIC58) Businesses"]) / opportunity_competition_df["2021 Total Population (Esri)"]

opportunity_competition_df["Pct of Food Related Businesses Per Capita"]

0            0.0
1       0.004119
2       0.005751
3       0.002694
4       0.012156
          ...   
2052    0.224444
2053    0.000555
2054    0.022036
2055         0.0
2056    0.032787
Name: Pct of Food Related Businesses Per Capita, Length: 2057, dtype: Float64

Projected population growth. Calculated by dividing the projected population in 2026 by the actual population in 2021. 

In [23]:
opportunity_competition_df["Projected Population Growth Rate"] = (opportunity_competition_df["2021 Total Population (Esri)"] 
                                                                  / opportunity_competition_df["2026 Total Population (Esri)"] -1)

opportunity_competition_df["Projected Population Growth Rate"] 

0       0.021765
1        0.02294
2       0.022876
3      -0.007353
4       0.022906
          ...   
2052    0.015801
2053   -0.036879
2054   -0.061084
2055   -0.032531
2056    0.020364
Name: Projected Population Growth Rate, Length: 2057, dtype: Float64

In [24]:
opportunity_competition_df

,fips,Avg 2022 Food,Avg 2022 Food Away from Home,Avg 2027 Food Away from Home,2021 Food & Beverage Stores (NAICS445) Businesses,2021 Food Srv & Drinking Places (NAICS722) Businesses,2021 Total (SIC01-99) Businesses,2021 Food Stores (SIC54) Businesses,2021 Eating & Drinking (SIC58) Businesses,2021 Diversity Index (Esri),2021 Median Disposable Income (Esri),2021 Population Density (Pop per Square Mile) (Esri),2026 Population Density (Pop per Square Mile) (Esri),2021 Total Population (Esri),2026 Total Population (Esri),Food Away From Home Spending Rate,Project 5Yr Food Away Home Spending Growth,Total Pct of Food Related Businesses,Pct of Food Related Businesses Per Capita,Projected Population Growth Rate
0,060730100101,6877.52,2942.05,3623.67,0.0,0.0,11.0,0.0,0.0,78.7,47338.0,15154.6,14831.8,1690.0,1654.0,0.427778,0.231682,0.0,0.0,0.021765
1,060730100102,8364.19,3457.18,4231.38,2.0,3.0,25.0,2.0,2.0,80.8,45231.0,9018.8,8816.6,2185.0,2136.0,0.413331,0.22394,0.36,0.004119,0.02294
2,060730100103,7101.43,2935.24,3277.84,1.0,3.0,29.0,2.0,3.0,84.8,46459.0,15980.3,15622.9,1565.0,1530.0,0.413331,0.11672,0.310345,0.005751,0.022876
3,060730100111,10566.02,4519.91,5287.83,1.0,1.0,16.0,1.0,1.0,84.2,75000.0,7335.9,7390.2,1485.0,1496.0,0.427778,0.169897,0.25,0.002694,-0.007353
4,060730150012,11587.27,4781.76,6792.55,2.0,8.0,43.0,1.0,8.0,73.1,59813.0,9046.8,8844.2,1563.0,1528.0,0.412674,0.420513,0.44186,0.012156,0.022906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2052,060730221012,20195.98,8544.28,9033.19,8.0,41.0,1594.0,12.0,40.0,58.4,113615.0,131.2,129.1,450.0,443.0,0.423068,0.057221,0.063363,0.224444,0.015801
2053,060730221021,22616.71,9568.42,11358.21,0.0,0.0,36.0,1.0,0.0,52.9,131072.0,3204.1,3326.8,1802.0,1871.0,0.423069,0.187052,0.027778,0.000555,-0.036879
2054,060730221022,22722.96,9613.37,11373.74,4.0,17.0,180.0,5.0,16.0,64.2,132157.0,3840.9,4090.8,1906.0,2030.0,0.423069,0.183117,0.233333,0.022036,-0.061084
2055,060730221023,14510.63,6183.54,7345.41,0.0,0.0,24.0,0.0,0.0,80.0,87061.0,4379.3,4526.5,2171.0,2244.0,0.426139,0.187897,0.0,0.0,-0.032531


#### Remove bias or unnecessary variables

In [25]:
opportunity_competition_df.drop(columns = ["Avg 2022 Food", "Avg 2022 Food Away from Home", "Avg 2027 Food Away from Home", 
                                          "2021 Food & Beverage Stores (NAICS445) Businesses", "2021 Food Srv & Drinking Places (NAICS722) Businesses",
                                          "2021 Total (SIC01-99) Businesses", "2021 Food Stores (SIC54) Businesses", "2021 Eating & Drinking (SIC58) Businesses",
                                          "2026 Population Density (Pop per Square Mile) (Esri)", "2021 Total Population (Esri)", "2026 Total Population (Esri)",
                                          "2021 Median Disposable Income (Esri)"], inplace = True)
opportunity_competition_df

,fips,2021 Diversity Index (Esri),2021 Population Density (Pop per Square Mile) (Esri),Food Away From Home Spending Rate,Project 5Yr Food Away Home Spending Growth,Total Pct of Food Related Businesses,Pct of Food Related Businesses Per Capita,Projected Population Growth Rate
0,060730100101,78.7,15154.6,0.427778,0.231682,0.0,0.0,0.021765
1,060730100102,80.8,9018.8,0.413331,0.22394,0.36,0.004119,0.02294
2,060730100103,84.8,15980.3,0.413331,0.11672,0.310345,0.005751,0.022876
3,060730100111,84.2,7335.9,0.427778,0.169897,0.25,0.002694,-0.007353
4,060730150012,73.1,9046.8,0.412674,0.420513,0.44186,0.012156,0.022906
...,...,...,...,...,...,...,...,...
2052,060730221012,58.4,131.2,0.423068,0.057221,0.063363,0.224444,0.015801
2053,060730221021,52.9,3204.1,0.423069,0.187052,0.027778,0.000555,-0.036879
2054,060730221022,64.2,3840.9,0.423069,0.183117,0.233333,0.022036,-0.061084
2055,060730221023,80.0,4379.3,0.426139,0.187897,0.0,0.0,-0.032531


Further clean up

In [26]:
opportunity_competition_df['2021 Diversity Index (Esri)'] = opportunity_competition_df['2021 Diversity Index (Esri)']/100

In [27]:
opportunity_competition_df.set_index('fips', inplace = True)

### Make the opportunity score

#### Scale the data

In [28]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

Standardize each column on a scale -1 to 1 based on the mean.

In [29]:
opportunity_competition_df[opportunity_competition_df.columns] = scaler.fit_transform(opportunity_competition_df)
opportunity_competition_df

,2021 Diversity Index (Esri),2021 Population Density (Pop per Square Mile) (Esri),Food Away From Home Spending Rate,Project 5Yr Food Away Home Spending Growth,Total Pct of Food Related Businesses,Pct of Food Related Businesses Per Capita,Projected Population Growth Rate
fips,,,,,,,
060730100101,0.501946,0.717180,0.947194,0.413864,-0.954059,-0.232773,0.495591
060730100102,0.660222,-0.004886,-0.523809,0.331707,1.451413,-0.136343,0.523990
060730100103,0.961701,0.814349,-0.523838,-0.806064,1.119624,-0.098141,0.522437
060730100111,0.916479,-0.202931,0.947211,-0.241767,0.716408,-0.169713,-0.208377
060730150012,0.079876,-0.001591,-0.590763,2.417647,1.998394,0.051814,0.523160
...,...,...,...,...,...,...,...
060730221012,-1.028057,-1.050786,0.467668,-1.437438,-0.530678,5.021704,0.351404
060730221021,-1.442590,-0.689165,0.467694,-0.059731,-0.768451,-0.219781,-0.922195
060730221022,-0.590913,-0.614225,0.467689,-0.101487,0.605044,0.283105,-1.507379


#### Make a score
Make the negative impact columns, negative. Such that the higher values will have a negative impact on the score. These columns include Total Pct of Food Related Businesses and Pct of Food Related Businesses Per Capita.
Add all the values together for the final score. 

In [30]:
opportunity_competition_df['Total Pct of Food Related Businesses'] = (-1)*opportunity_competition_df['Total Pct of Food Related Businesses']
opportunity_competition_df['Pct of Food Related Businesses Per Capita'] = (-1)*opportunity_competition_df['Pct of Food Related Businesses Per Capita']

Cap the total STD for [Project 5Yr Food Away Home Spending Growth] to 4 STD (major outlier is skewing the final score).

In [31]:
opportunity_competition_df.describe()

,2021 Diversity Index (Esri),2021 Population Density (Pop per Square Mile) (Esri),Food Away From Home Spending Rate,Project 5Yr Food Away Home Spending Growth,Total Pct of Food Related Businesses,Pct of Food Related Businesses Per Capita,Projected Population Growth Rate
count,2.057000e+03,2.057000e+03,2.053000e+03,2.053000e+03,2.057000e+03,2.057000e+03,2057.000000
mean,-7.184876e-16,-1.658048e-16,2.630358e-16,-5.883695e-17,2.072560e-17,-1.036280e-17,0.000000
std,1.000243e+00,1.000243e+00,1.000244e+00,1.000244e+00,1.000243e+00,1.000243e+00,1.000243
min,-4.419689e+00,-1.066037e+00,-5.053605e+00,-4.375874e+00,-5.727808e+00,-2.986713e+01,-13.914076
25%,-6.210609e-01,-6.195329e-01,-6.834765e-01,-4.050449e-01,-5.308007e-01,1.970318e-02,-0.030612
50%,1.853938e-01,-2.137930e-01,-1.325965e-02,-5.695090e-02,2.507042e-01,1.774277e-01,0.409540
75%,8.637202e-01,3.180419e-01,9.012543e-01,3.557510e-01,9.540585e-01,2.327729e-01,0.501511
max,1.391307e+00,1.139741e+01,2.916880e+00,2.705140e+01,9.540585e-01,2.327729e-01,0.934502


Cap the outliers at 4/-4 so extreme values don't skew the final score.

In [32]:
def truncate_std(std, boundary = 4):
    if std > boundary:
        return boundary
    elif std < -boundary:
        return -boundary
    else:
        return std

In [33]:
for col in opportunity_competition_df.columns:
    opportunity_competition_df[col] = opportunity_competition_df[col].apply(lambda x: truncate_std(x))

In [34]:
opportunity_competition_df['Opportunity_Competition_Score'] = opportunity_competition_df.sum(axis=1)
opportunity_competition_df['Opportunity_Competition_Score'].describe()

count    2057.000000
mean        0.023140
std         2.452716
min       -10.698047
25%        -1.608548
50%         0.075404
75%         1.739337
max         7.677773
Name: Opportunity_Competition_Score, dtype: float64

#### Scale the opportunity score to the min/max values so it ranges 0 to 1

In [35]:
scoremin= opportunity_competition_df['Opportunity_Competition_Score'].min()
scoremax = opportunity_competition_df['Opportunity_Competition_Score'].max()

opportunity_competition_df['Opportunity_Competition_Score_Scaled'] = ((opportunity_competition_df['Opportunity_Competition_Score']-scoremin) 
                                                                      / (scoremax-scoremin) )

opportunity_competition_df['Opportunity_Competition_Score_Scaled']

fips
060730100101    0.814149
060730100102    0.564339
060730100103    0.579302
060730100111    0.618311
060730150012    0.602758
                  ...   
060730221012    0.246602
060730221021    0.491967
060730221022    0.406163
060730221023    0.644672
060730221024    0.649228
Name: Opportunity_Competition_Score_Scaled, Length: 2057, dtype: float64

In [36]:
opportunity_competition_df['Opportunity_Competition_Score_Scaled'].describe()

count    2057.000000
mean        0.583440
std         0.133475
min         0.000000
25%         0.494645
50%         0.586284
75%         0.676834
max         1.000000
Name: Opportunity_Competition_Score_Scaled, dtype: float64

In [37]:
opportunity_competition_df.describe()

,2021 Diversity Index (Esri),2021 Population Density (Pop per Square Mile) (Esri),Food Away From Home Spending Rate,Project 5Yr Food Away Home Spending Growth,Total Pct of Food Related Businesses,Pct of Food Related Businesses Per Capita,Projected Population Growth Rate,Opportunity_Competition_Score,Opportunity_Competition_Score_Scaled
count,2057.000000,2057.000000,2053.000000,2053.000000,2057.000000,2057.000000,2057.000000,2057.000000,2057.000000
mean,0.000204,-0.016175,0.004434,-0.011434,0.002551,0.027602,0.015945,0.023140,0.583440
std,0.999384,0.900347,0.980071,0.804173,0.988661,0.449498,0.891849,2.452716,0.133475
min,-4.000000,-1.066037,-4.000000,-4.000000,-4.000000,-4.000000,-4.000000,-10.698047,0.000000
25%,-0.621061,-0.619533,-0.683477,-0.405045,-0.530801,0.019703,-0.030612,-1.608548,0.494645
50%,0.185394,-0.213793,-0.013260,-0.056951,0.250704,0.177428,0.409540,0.075404,0.586284
75%,0.863720,0.318042,0.901254,0.355751,0.954059,0.232773,0.501511,1.739337,0.676834
max,1.391307,4.000000,2.916880,4.000000,0.954059,0.232773,0.934502,7.677773,1.000000


#### Export final result

In [38]:
opportunity_competition_df.reset_index(inplace = True, drop = False)

In [39]:
opportunity_competition_df[['fips', 'Opportunity_Competition_Score_Scaled']].to_csv('Fips_Opportunity_Competition_Score.csv', index = False)